In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz, process
import random
import numpy as np
from IPython.display import display

In [ ]:
df = pd.read_csv('Zara_Kaz_Functions.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 18 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   name                             1112 non-null   object
 1   sku                              1112 non-null   object
 2   brand                            1112 non-null   object
 3   description                      1112 non-null   object
 4   price                            1112 non-null   int64 
 5   availability                     1112 non-null   object
 6   condition                        1112 non-null   object
 7   color                            1112 non-null   object
 8   size                             1112 non-null   object
 9   event_category                   1112 non-null   object
 10  category                         1112 non-null   object
 11  scope_(inscope/outofscope)       1112 non-null   object
 12  url                              1

In [ ]:
df.head()

name  \
0           LONG PRINT DRESS   
1          PLAID SHIRT DRESS   
2               CORDUROY CAP   
3            CROPPED T-SHIRT   
4  GEOMETRIC PRINT OVERSHIRT   

                                         description  price availability  \
0  Long dress with round neck and long sleeves. B...     49        empty   
1  Shirtdress with lapel collar and long sleeves....     27        empty   
2               Cap with visor. Back fit adjustment.     25      InStock   
3           Sleeveless crop top with round neckline.      7        empty   
4  Relaxed fit overshirt with lapel collar and lo...     69      InStock   

    color size            event_category   category  \
0     RED    L  Casual and Everyday Wear      DRESS   
1  YELLOW    L  Casual and Everyday Wear      DRESS   
2  YELLOW    S        Beach and Vacation        CAP   
3    BLUE    S    Evening and Party Wear    T-SHIRT   
4     RED    M      Sleep and Loungewear  OVERSHIRT   

                                                 url price_category  \
0  https://www.zara.com/us/en/long-print-dress-p0...         MEDIUM   
1  https://www.zara.com/us/en/plaid-shirt-dress-p...            LOW   
2  https://www.zara.com/us/en/corduroy-cap-p09065...            LOW   
3  https://www.zara.com/us/en/cropped-t-shirt-p04...            LOW   
4  https://www.zara.com/us/en/geometric-print-ove...         MEDIUM   

  complimentary_clothing_category deadstock  popularity_score  \
0                             TOP    emplty                72   
1                        LEGGINGS    emplty                43   
2                            HOME    emplty                45   
3                             CAP    emplty                45   
4                           DRESS   InStock                59   

   trendiness_score  
0                52  
1                94  
2                87  
3                91  
4                71

In [ ]:

df = df[['name', 'description', 'price', 'availability', 'color', 'size', 'event_category', 'category',
        'url', 'price_category',
       'complimentary_clothing_category', 'deadstock', 'popularity_score',
       'trendiness_score']]

In [ ]:
# future improvement: customer segmentation based on purchase history 
# availability and deadstock data are synced with real time data

### Data Preprocessing 

In [ ]:
# df = pd.read_csv('ZaraData.csv')
df = df.drop('complimentary_clothing_category', axis=1)

df = df[df['scope_(inscope/outofscope)'] == 'INSCOPE']

# color - randomly assign 
# Create a list of colors
colors = ['RED', 'GREEN', 'BLUE', 'YELLOW']

# replace column 'color' and randomly assign colors
df['color'] = [random.choice(colors) for _ in range(len(df))]
df.color.unique()

sizes = ['S', 'M', 'L']

# replace 'size' and randomly assign colors
df['size_list'] = [random.choice(sizes) for _ in range(len(df))]
df = df.rename({'size_list':'size'}, axis=1)


# replace event column
events = [
    'Casual and Everyday Wear',
    'Formal and Business Events',
    'Evening and Party Wear',
    'Beach and Vacation',
    'Athletic and Workout',
    'Sleep and Loungewear',
    'Winter and Cold Weather',
    'Summer and Hot Weather',
    'Festive and Holiday Wear'
]

df['event_category'] = [random.choice(events) for _ in range(len(df))]

# price category
df.price = df.price.astype('int')
bin_edges = [0, 40, 80, 120, df['price'].max() + 1]
labels = ['LOW', 'MEDIUM', 'HIGH', 'LUXURY']
# Categorize the 'Price' column based on bins and labels
df['price_category'] = pd.cut(df['price'], bins=bin_edges, labels=labels, right=False)

# complimentary column 

complimentary = ['HOME', 'DRESS', 'CAP', 'T-SHIRT', 'OVERSHIRT', 'SHOES', 'TOP',
       'PANTS', 'SHIRT', 'BAGS', 'BELT', 'JACKET', 'SHORTS', 'LEGGINGS',
       'BOOTS', 'SKIRT', 'JEANS']

df['complimentary_clothing_category'] = [random.choice(complimentary) for _ in range(len(df))]

for index, row in df.iterrows():
    df.loc[index, 'complimentary_clothing_category'] = random.choice([c for c in complimentary if c != row['category']])

#categorical
df['color'] = pd.Categorical(df['color'])
df['size'] = pd.Categorical(df['size'], categories=['S', 'M', 'L'], ordered=True)
df['price_category'] = pd.Categorical(df['price_category'], categories=['LOW', 'MEDIUM', 'HIGH', 'LUXURY'], ordered=True)



df['popularity_score'] = np.random.randint(40, 100, size=len(df))
df['trendiness_score'] = np.random.randint(40, 100, size=len(df))


In [ ]:
df = pd.read_csv('Zara_Kaz_Functions.csv')

In [ ]:
df['popularity_score'] = np.random.randint(40, 100, size=len(df))
df['trendiness_score'] = np.random.randint(40, 100, size=len(df))

In [ ]:
df.head()

name               sku brand  \
0           LONG PRINT DRESS   142448983-701-1  ZARA   
1          PLAID SHIRT DRESS  138550324-800-18  ZARA   
2               CORDUROY CAP   131213214-807-3  ZARA   
3            CROPPED T-SHIRT   128032853-800-1  ZARA   
4  GEOMETRIC PRINT OVERSHIRT   142882821-427-2  ZARA   

                                         description  price availability  \
0  Long dress with round neck and long sleeves. B...     49        empty   
1  Shirtdress with lapel collar and long sleeves....     27        empty   
2               Cap with visor. Back fit adjustment.     25      InStock   
3           Sleeveless crop top with round neckline.      7        empty   
4  Relaxed fit overshirt with lapel collar and lo...     69      InStock   

      condition   color size            event_category   category  \
0  NewCondition     RED    L  Casual and Everyday Wear      DRESS   
1  NewCondition  YELLOW    L  Casual and Everyday Wear      DRESS   
2  NewCondition  YELLOW    S        Beach and Vacation        CAP   
3  NewCondition    BLUE    S    Evening and Party Wear    T-SHIRT   
4  NewCondition     RED    M      Sleep and Loungewear  OVERSHIRT   

  scope_(inscope/outofscope)  \
0                    INSCOPE   
1                    INSCOPE   
2                    INSCOPE   
3                    INSCOPE   
4                    INSCOPE   

                                                 url price_category  \
0  https://www.zara.com/us/en/long-print-dress-p0...         MEDIUM   
1  https://www.zara.com/us/en/plaid-shirt-dress-p...            LOW   
2  https://www.zara.com/us/en/corduroy-cap-p09065...            LOW   
3  https://www.zara.com/us/en/cropped-t-shirt-p04...            LOW   
4  https://www.zara.com/us/en/geometric-print-ove...         MEDIUM   

  complimentary_clothing_category deadstock  popularity_score  \
0                             TOP    emplty                72   
1                        LEGGINGS    emplty                43   
2                            HOME    emplty                45   
3                             CAP    emplty                45   
4                           DRESS   InStock                59   

   trendiness_score  
0                52  
1                94  
2                87  
3                91  
4                71

In [ ]:
df.to_csv('Zara_Kaz_Functions.csv', index=False)

### Event Based Rec. Func.

In [ ]:
df = pd.read_csv('Zara_Kaz_Functions.csv')

In [ ]:
df.head()

name               sku brand  \
0           LONG PRINT DRESS   142448983-701-1  ZARA   
1          PLAID SHIRT DRESS  138550324-800-18  ZARA   
2               CORDUROY CAP   131213214-807-3  ZARA   
3            CROPPED T-SHIRT   128032853-800-1  ZARA   
4  GEOMETRIC PRINT OVERSHIRT   142882821-427-2  ZARA   

                                         description  price availability  \
0  Long dress with round neck and long sleeves. B...     49        empty   
1  Shirtdress with lapel collar and long sleeves....     27        empty   
2               Cap with visor. Back fit adjustment.     25      InStock   
3           Sleeveless crop top with round neckline.      7        empty   
4  Relaxed fit overshirt with lapel collar and lo...     69      InStock   

      condition   color size            event_category   category  \
0  NewCondition     RED    L  Casual and Everyday Wear      DRESS   
1  NewCondition  YELLOW    L  Casual and Everyday Wear      DRESS   
2  NewCondition  YELLOW    S        Beach and Vacation        CAP   
3  NewCondition    BLUE    S    Evening and Party Wear    T-SHIRT   
4  NewCondition     RED    M      Sleep and Loungewear  OVERSHIRT   

  scope_(inscope/outofscope)  \
0                    INSCOPE   
1                    INSCOPE   
2                    INSCOPE   
3                    INSCOPE   
4                    INSCOPE   

                                                 url price_category  \
0  https://www.zara.com/us/en/long-print-dress-p0...         MEDIUM   
1  https://www.zara.com/us/en/plaid-shirt-dress-p...            LOW   
2  https://www.zara.com/us/en/corduroy-cap-p09065...            LOW   
3  https://www.zara.com/us/en/cropped-t-shirt-p04...            LOW   
4  https://www.zara.com/us/en/geometric-print-ove...         MEDIUM   

  complimentary_clothing_category  
0                             TOP  
1                        LEGGINGS  
2                            HOME  
3                             CAP  
4                           DRESS

In [ ]:
df[df['category'] == df['complimentary_clothing_category']]

Empty DataFrame
Columns: [name, sku, brand, description, price, availability, condition, color, size, event_category, category, scope_(inscope/outofscope), url, price_category, complimentary_clothing_category]
Index: []

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1112 entries, 0 to 1111
Data columns (total 15 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   name                             1112 non-null   object
 1   sku                              1112 non-null   object
 2   brand                            1112 non-null   object
 3   description                      1112 non-null   object
 4   price                            1112 non-null   int64 
 5   availability                     1112 non-null   object
 6   condition                        1112 non-null   object
 7   color                            1112 non-null   object
 8   size                             1112 non-null   object
 9   event_category                   1112 non-null   object
 10  category                         1112 non-null   object
 11  scope_(inscope/outofscope)       1112 non-null   object
 12  url                              1

In [ ]:
instock_indices = df[df['availability'] == 'InStock'].index

# Calculate the number of rows to assign True to (approximately 25%)
num_rows_to_assign_true = int(0.25 * len(instock_indices))

# Randomly select indices to assign True to
random_indices = random.sample(instock_indices.tolist(), num_rows_to_assign_true)

# Assign Instock to the selected indices, and False to the rest
df.loc[random_indices, 'deadstock'] = 'InStock'
df['deadstock'].fillna('emplty', inplace=True)  # empty remaining rows with False

In [ ]:
# df.to_csv('Zara_Kaz_Functions.csv', index=False)e

In [ ]:
# for testing more conversational version 
# fix case sensitivity 
# price range order is wrong..? maybe just delete luxury 
# show the result one by one for users (name, size, color, url... ) 

# take an input "deadstock" boolean,  then push that product in recommnedation(just filtering) 
from fuzzywuzzy import fuzz, process

def match_event(event):
    # List of event categories
    event_lists = [
        'Casual and Everyday Wear',
        'Formal and Business Events',
        'Evening and Party Wear',
        'Beach and Vacation',
        'Athletic and Workout',
        'Sleep and Loungewear',
        'Winter and Cold Weather',
        'Summer and Hot Weather',
        'Festive and Holiday Wear'
    ]
    # Find the best-matched event category
    matched_event, score = process.extractOne(event.lower(), event_lists, scorer=fuzz.partial_ratio)
    return matched_event, score

def filter_recommendations(recommended_items, user_color_preference, user_size_preference, user_price_preference):
    # Filter recommended items based on user preferences
    filtered_items = recommended_items[
        (recommended_items['color'] == user_color_preference) &
        (recommended_items['size'] == user_size_preference) &
        (recommended_items['price_category'] == user_price_preference)
    ]
    return filtered_items

def find_complimentary_items(clothing_data, recommended_item, matched_event, user_color_preference, user_size_preference):
    # Find items that complement the recommended item
    recommended_category = recommended_item['category'].item()
    complimentary_category = recommended_item['complimentary_clothing_category'].item()

    complimentary_items = clothing_data[
        (clothing_data['category'] == complimentary_category) &
        (clothing_data['event_category'] == matched_event) &
        (clothing_data['color'] == user_color_preference) &
        (clothing_data['size'] == user_size_preference) &
        (clothing_data['availability'] == 'InStock')
    ]
    return complimentary_items

def event_based_recommendation(event, clothing_data):
    print("Hello there! Let's find the perfect outfit for your event.")
    matched_event, score = match_event(event)

    if score >= 70:
        print(f"Great! It looks like you're attending a '{matched_event}' event.")
        
        recommended_items = clothing_data[clothing_data['event_category'] == matched_event]
        recommended_items = recommended_items[recommended_items['availability'] == 'InStock']

        available_colors = recommended_items['color'].unique()
        available_sizes = recommended_items['size'].unique()
        available_price_range = recommended_items['price_category'].unique().tolist()

        # Gather user preferences
        user_color_preference = input(f"Available colors: {', '.join(available_colors)}. What's your preferred color? ").upper()
        user_size_preference = input(f"Available sizes: {', '.join(available_sizes)}. What's your preferred size? ").upper()
        print('Price Range: low: < $40, medium: < $80, high < $120, luxury +$120')
        user_price_preference = input(f"Available price range: {', '.join(available_price_range)}. What's your preferred price range? ").upper()

        recommended_items = filter_recommendations(recommended_items, user_color_preference, user_size_preference, user_price_preference)

        if len(recommended_items) > 0:
            recommended_item = recommended_items.sample(n=1)
            complimentary_item = find_complimentary_items(clothing_data, recommended_item, matched_event, user_color_preference, user_size_preference)

            if len(complimentary_item) >= 1:
                complimentary_item = complimentary_item.sample(n=1)

                print('Here is our recommendation:')
                print(f"Name: {recommended_item['name'].values[0]}")
                print(f"Size: {recommended_item['size'].values[0]}")
                print(f"Color: {recommended_item['color'].values[0]}")
                print(f"Price: {recommended_item['price'].values[0]}")
                print(f"URL: {recommended_item['url'].values[0]}")
                
                print('\nYou might also be interested in this:')
                print(f"Name: {complimentary_item['name'].values[0]}")
                print(f"Size: {complimentary_item['size'].values[0]}")
                print(f"Color: {complimentary_item['color'].values[0]}")
                print(f"Price: {complimentary_item['price'].values[0]}")
                print(f"URL: {complimentary_item['url'].values[0]}")

            else:
                print('Here is our recommendation:')
                print(f"Name: {recommended_item['name'].values[0]}")
                print(f"Size: {recommended_item['size'].values[0]}")
                print(f"Color: {recommended_item['color'].values[0]}")
                print(f"Price: {recommended_item['price'].values[0]}")
                print(f"URL: {recommended_item['url'].values[0]}")
                
        else: 
            print("I'm sorry, I couldn't find suitable recommendations for that event with your preferences.")
    else:
        print("I'm sorry, I couldn't find suitable recommendations for that event.")

# Example usage
clothing_data = df  # Replace with your actual clothing data
event = input("Hello! What's the occasion you're dressing up for? ")
event_based_recommendation(event, clothing_data)

# this is a scenario where complimentray works:   casual, blue, size L, price medium

Hello there! Let's find the perfect outfit for your event.
Great! It looks like you're attending a 'Casual and Everyday Wear' event.
Price Range: low: < $40, medium: < $80, high < $120, luxury +$120
Here is our recommendation:
Name: LEATHER SNEAKERS
Size: L
Color: BLUE
Price: 55
URL: https://www.zara.com/us/en/leather-sneakers-p14305830.html

You might also be interested in this:
Name: METALLIC HEELED ANKLE BOOTS
Size: L
Color: BLUE
Price: 59
URL: https://www.zara.com/us/en/metallic-heeled-ankle-boots-p13108710.html


### Hot items recc

In [ ]:
df = pd.read_csv('ZaraData.csv')
df = df[df['scope_(inscope/outofscope)'] == 'INSCOPE']

In [ ]:
df.head()

name               sku brand  \
1           LONG PRINT DRESS   142448983-701-1  ZARA   
2          PLAID SHIRT DRESS  138550324-800-18  ZARA   
3               CORDUROY CAP   131213214-807-3  ZARA   
5            CROPPED T-SHIRT   128032853-800-1  ZARA   
7  GEOMETRIC PRINT OVERSHIRT   142882821-427-2  ZARA   

                                         description  price availability  \
1  Long dress with round neck and long sleeves. B...     49        empty   
2  Shirtdress with lapel collar and long sleeves....     27        empty   
3               Cap with visor. Back fit adjustment.     25      InStock   
5           Sleeveless crop top with round neckline.      7        empty   
7  Relaxed fit overshirt with lapel collar and lo...     69      InStock   

      condition   color size            event_category   category  \
1  NewCondition  Yellow    L        Beach and Vacation      DRESS   
2  NewCondition     Red    M      Athletic and Workout      DRESS   
3  NewCondition     Red    M  Festive and Holiday Wear        CAP   
5  NewCondition     Red    S    Evening and Party Wear    T-SHIRT   
7  NewCondition  Yellow    M  Festive and Holiday Wear  OVERSHIRT   

  complimentary_clothing_category scope_(inscope/outofscope)  \
1                           PANTS                    INSCOPE   
2                            POLO                    INSCOPE   
3                           JEANS                    INSCOPE   
5                        JUMPSUIT                    INSCOPE   
7                       JEWELLERY                    INSCOPE   

                                                 url Price_Category  \
1  https://www.zara.com/us/en/long-print-dress-p0...         medium   
2  https://www.zara.com/us/en/plaid-shirt-dress-p...            low   
3  https://www.zara.com/us/en/corduroy-cap-p09065...            low   
5  https://www.zara.com/us/en/cropped-t-shirt-p04...            low   
7  https://www.zara.com/us/en/geometric-print-ove...         medium   

   popularity_score  trendiness_score  
1                85                80  
2                94                98  
3                91                51  
5                96                89  
7                63                91

In [ ]:
df = df

def get_user_preferences():
    print("Let's get started! Please provide your preferences:")
    
    user_category_preference = input("Which category of hot items are you looking for? ")
    user_color_preference = input("What color do you prefer for your outfit? ")
    user_size_preference = input("What size do you wear? ")
    user_price_preference = input("What's your preferred price category? (Low, Medium, High): ")
    
    return user_category_preference, user_color_preference, user_size_preference, user_price_preference

def trendy_item_recommendation(user_category_preference, user_color_preference, user_size_preference, user_price_preference):
    print("Hi there! Let's find you some trendy items that match your preferences.")

    # Filter items based on user preferences
    filtered_items = df[
        (df['category'] == user_category_preference) &
        (df['color'] == user_color_preference) &
        (df['size'] == user_size_preference) &
        (df['Price_Category'] <= user_price_preference) &
        (df['availability'] == 'InStock')
    ]

    if len(filtered_items) > 0:
        sorted_items = filtered_items.sort_values(by=['popularity_score', 'trendiness_score'], ascending=[False, False])

        if len(sorted_items) > 0:
            recommended_item = sorted_items.iloc[0]
            print(f"I recommend '{recommended_item['name']}' by {recommended_item['brand']}!")
            print(f"Description: {recommended_item['description']}")
            print(f"Price: ${recommended_item['price']}")
            print(f"Category: {recommended_item['category']}")
            print(f"You can check it out [here]({recommended_item['url']}).")
        else:
            print("I'm sorry, I couldn't find any trendy items matching your preferences.")
    else:
        print("I'm sorry, I couldn't find any items matching your preferences.")

# Example usage
user_category_preference, user_color_preference, user_size_preference, user_price_preference = get_user_preferences()
trendy_item_recommendation(user_category_preference, user_color_preference, user_size_preference, user_price_preference)


Let's get started! Please provide your preferences:
Hi there! Let's find you some trendy items that match your preferences.
I recommend 'PLEATED DRESS' by ZARA!
Description: Round neck short sleeve dress. Pleat detail at skirt.
Price: $35
Category: DRESS
You can check it out [here](https://www.zara.com/us/en/pleated-dress-p00264869.html).


In [ ]:
# from fuzzywuzzy import process, fuzz

# def match_event(event):
#     event_lists = [
#         'Casual and Everyday Wear',
#         'Formal and Business Events',
#         'Evening and Party Wear',
#         'Beach and Vacation',
#         'Athletic and Workout',
#         'Sleep and Loungewear',
#         'Winter and Cold Weather',
#         'Summer and Hot Weather',
#         'Festive and Holiday Wear'
#     ]
#     matched_event, score = process.extractOne(event.lower(), event_lists, scorer=fuzz.partial_ratio)
#     return matched_event, score

# def filter_recommendations(recommended_items, user_color_preference, user_size_preference, user_price_preference):
#     filtered_items = recommended_items[
#         (recommended_items['color'] == user_color_preference) &
#         (recommended_items['size'] == user_size_preference) &
#         (recommended_items['Price_Category'] == user_price_preference)
#     ]
#     return filtered_items

# def find_complimentary_items(clothing_data, recommended_item, matched_event, user_color_preference, user_size_preference):
#     recommended_category = recommended_item['category'].item()
#     complimentary_category = recommended_item['complimentary_clothing_category'].item()

#     complimentary_items = clothing_data[
#         (clothing_data['category'] == complimentary_category) &
#         (clothing_data['event_category'] == matched_event) &
#         (clothing_data['color'] == user_color_preference) &
#         (clothing_data['size'] == user_size_preference) &
#         (clothing_data['availability'] == 'InStock')
#     ]

#     return complimentary_items

# def event_based_recommendation(event, clothing_data):
#     matched_event, score = match_event(event)

#     if score >= 80: 
#         recommended_items = clothing_data[clothing_data['event_category'] == matched_event]
#         recommended_items = recommended_items[recommended_items['availability'] == 'InStock']

#         available_colors = recommended_items['color'].unique()
#         available_sizes = recommended_items['size'].unique()
#         available_price_range = recommended_items['Price_Category'].unique().tolist()
#         user_color_preference = input(f"Available colors: {', '.join(available_colors)}. Enter preferred color: ")
#         user_size_preference = input(f"Available sizes: {', '.join(available_sizes)}. Enter preferred size: ")
#         print('Price Range: low: < 40, medium: < 80, high < 120, luxury: <120')
#         user_price_preference = input(f"Available price range: {', '.join(available_price_range)}. Enter preferred price: ")

#         recommended_items = filter_recommendations(recommended_items, user_color_preference, user_size_preference, user_price_preference)

#         if len(recommended_items) > 0:
#             recommended_item = recommended_items.sample(n=1)
#             complimentary_items = find_complimentary_items(clothing_data, recommended_item, matched_event, user_color_preference, user_size_preference)

#             if len(complimentary_items) >= 2:
#                 complimentary_item_samples = complimentary_items.sample(n=2)
#             else:
#                 complimentary_item_samples = complimentary_items

#             return recommended_item, complimentary_item_samples
        
#         return None, "Sorry, I couldn't find suitable recommendations for that event with your preference."
    
#     return None, "Sorry, I couldn't find suitable recommendations for that event."

# # Example usage
# clothing_data = df
# event = input("Enter the event: ")
# recommended_item, complimentary_items = event_based_recommendation(event, clothing_data)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=efa188a0-2f94-4804-9f19-625dc30c6533' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>